# Building a Feedforward Neural Network Model
This notebook provides instructions on how to build a feedforward neural network model. It will introduce a couple of classes defined in the [FFBP package](https://github.com/alex-ten/pdpyflow/tree/master/FFBP): `FFBP.Layer` and `FFBP.Model` as well as describe some Tensorflow objects needed to successfully implement a feedforward neural network model with `FFBP`.

To demonstrate the various ways in which FFBP layers can be connected together, our examle will be the [semantic cognition](https://stanford.edu/~jlmcc/papers/RogersMcC08BBSFinalProof.pdf) by Rogers and McClelland. The model is a feedforward neural network with two input layers, two hidden layers, and an output layer (as shown in the figure below).

<img src="materials/semantic_network.png" width=60%>

Based on these interconnected layers we will construct a working neural network model.

We begin by importing the packages required for this tutorial and creating a [Tensorflow Graph](https://www.tensorflow.org/programmers_guide/graphs). We want to make sure that we are adding network elements in the context of this graph (see cells below).

In [ ]:
import tensorflow as tf
import FFBP

FFBP_GRAPH = tf.Graph()

## Placeholders

Input layers are implemented as [tf.placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) objects. `InputData` (covered in a [different tutorial](https://github.com/alex-ten/pdpyflow/blob/master/tutorials/building_models/prepare_input.ipynb)) data from a text document into the `tf.float32` format. Therefore, the first argument inside the `tf.placeholder()` call is `dtype = tf.float32` (but it can be anything as long as it is consistent with the data that is actually fed into it). The `shape` parameter is optional, but we strictly indicate *at least* the second dimension because other objects in the software expect an integer value there. The `None` in the place of the first dimension of the `shape` parameter means that any size of the that dimension is allowed. For instance, `inp_1` is a placeholder for a 2-dimensional data tensor (of type `tf.float32`) with 3 columns and any number of rows. This is useful when we train a model with batched up input but test it on single examples.

The target placeholder needs to be the same size as the final (output) layer of the network. It is a good practice to name tensorflow nodes informatively, so we add an optional `name` argument accordingly.

In [ ]:
with FFBP_GRAPH.as_default():
    MODEL_NAME = 'semantic_net'
    with tf.name_scope(MODEL_NAME):
        
        # Create input placeholders
        ITEM = tf.placeholder(dtype = tf.float32, shape=[None, 3], name='item_input')
        REL  = tf.placeholder(dtype = tf.float32, shape=[None, 2], name='relation_input')

        # Create a target placeholder
        TARG = tf.placeholder(dtype = tf.float32, shape=[None, 8], name='target')

## Hidden and output layers
The transforming layers are created via the `FFBP.BasicLayer` interface. The `FFBP.BasicLayer` constructor takes the following arguments:
- **`layer_name`** : a (string) name of the layer (this will be displayed in visualization)
- **`layer_input`** : input to the layer. If input is a placeholder, just pass the corresponding variable handle to this parameter. If input comes from another `BasicLayer` object, we need to point to its `output` attribute (e.g. `hidden_layer.output`). Finally, if there are multiple layers / placeholders feeding into a single `BasicLayer` simply wrap the inputs into a list or a tuple (e.g. `(inp_1, inp_2, ..., inp_N)`)
- **`size`** : the number of units in the layer
- **`wrange`** : a (listed or tupled) pair of values corresponding to, respectively, the lower and upper bounds of the distribution from which weight values will be sampled uniformly upon initialization.
- **`nonlin`** : (*`default`*`= None`) a function that takes in and outputs a tensor. If omitted, input to the layer will be transformed linearly (i.e. $Wx + b$). Tensorflow provides a number of widely used nonlinear functions documented [here](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn/activation_functions_).
- **`seed`** : (*`default`*`= None`) the seed for random weight initialization. If given a negative value or omited, the initialization will be irreproducible.

`FFBP.BasicLayer`s are modular and can be configured with different sizes, nonlinearities, weight values etc.

In [ ]:
with FFBP_GRAPH.as_default():
    with tf.name_scope(MODEL_NAME):

        REPR = FFBP.BasicLayer(
            layer_name = 'representation_layer', 
            layer_input = ITEM, 
            size = 8, 
            wrange = [-.1, .1], 
            nonlin = tf.nn.sigmoid, 
        )
        
        HID = FFBP.BasicLayer(
            layer_name = 'hidden_layer', 
            layer_input = (REPR.output, REL), 
            size = 15, 
            wrange = [-.5, .5], 
            nonlin = None,
        )
        
        ATTR = FFBP.BasicLayer(
            layer_name = 'attribute_layer', 
            layer_input = HID.output, 
            size = 5, 
            wrange = [-.01, .02], 
            nonlin = tf.nn.sigmoid
        )

## FFBP Model
We can now create a feedforward model by instantiating an `FFBP.Model` class. The class constructor takes the following arguments:
- **`name`** : the name of the model (used for storage and visualization).
- **`loss`** : the loss (objective) function of the model.
- **`optimizer`** : model optimizer. This can be provided separately with `train_setup` or `set_optimizer` method. It is possible to write your own optimizers, but Tensorflow includes many widely used [optimizers](https://www.tensorflow.org/api_guides/python/train), which we use throughout the examples and tutorials in pdpyflow.
- **`layers`** : a list of model layers. The order is consequential for visualization. The last layer should be the output layer so that the viewer displays target output information for this layer, but not the others.
- **`inp`** : input placeholder or a list (or tuple) of input placeholders. The order of placeholders must correspond to the order of `inp_size` values set for `FFBP.InputData` (see the [input data tutorial](https://github.com/alex-ten/pdpyflow/blob/master/tutorials/building_models/prepare_input.ipynb)).
- **`train_data`** : `(`*`default`*`= None)` An instance of `FFBP.InputData` set up for training.
- **`test_data`** : `(`*`default`*`= None)` An instance of `FFBP.InputData` set up for testing.



In [ ]:
with FFBP_GRAPH.as_default():
    
    MODEL = FFBP.Model(
        name = MODEL_NAME,
        layers = [REPR, HID, ATTR], 
        inp    = [ITEM, REL],
        targ   = TARG,
        loss   = tf.squared_difference(TARG, OUT.output, name='loss_function')
    )

Training and testing can also be set up separately:
- **`test_setup`**`(`*`data`*`)` : sets up the model for testing separately from class initialization.
- **`train_setup`**`(`*`data, optimizer=None`*`)` : sets up the model for training separately from class initialization.

Finally, there are two special methods to actually run an epoch of training or testing. 
- **`test_epoch`**`(`*`session, verbose=False`*`)`: runs a single epoch of testing by feeding each item from the test set into the model and evaluating its state. This method returns two values: the loss accumulated across test items, and the model snapshot which contains information about the model state at the time of test.  If `verbose` is `True`, the loss value will be printed to the output along with the corresponding `Epoch`.
- **`train_epoch`**`(`*`session, verbose=False`*`)` : runs a single epoch of training, optimizing parameters after processing each mini-batch of training examples. Returns loss accumulated over input mini-batches. If `verbose` is `True`, this value will be printed to the output along with the corresponding `Epoch`.

## Next steps

The main purpose of this tutorial was to explain how a model is constructed with the FFBP package and Tensorflow. The next step is to actually run your model by training it on some data. Go to the [main tutorial](https://github.com/alex-ten/pdpyflow/blob/master/tutorials/building_models/main_tutorial.ipynb) to see how this can be approached.